In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import matplotlib.pyplot as plt

In [2]:
from __future__ import print_function
import keras
from keras.models import load_model
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.callbacks import TensorBoard
from keras import applications
from keras.models import Model, load_model
import numpy as np
from sklearn.metrics import confusion_matrix

from keras_applications.efficientnet import preprocess_input

from datetime import datetime


Using TensorFlow backend.


In [3]:
# Data generators, data augmentation
def preprocessing_f(x):
    return preprocess_input(x, data_format=None, backend=keras.backend, layers=keras.layers, models=keras.models, utils=keras.utils)

train = ImageDataGenerator(
    rotation_range = 90.0,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest',
    preprocessing_function = preprocessing_f)

valid = ImageDataGenerator(
    fill_mode = 'nearest',
    preprocessing_function = preprocessing_f)

In [4]:
# Target directories 
batch_size = 32
img_size = 300

trainGenerator = train.flow_from_directory('/raid/wanfang/Documents/simple_recyclable_classification/train', target_size = (img_size, img_size), batch_size=batch_size,
    class_mode = "categorical", color_mode = "rgb", shuffle=True, seed=42)

validationGenerator = valid.flow_from_directory('/raid/wanfang/Documents/simple_recyclable_classification/test', target_size = (img_size, img_size), batch_size=batch_size,
    class_mode = "categorical", color_mode = "rgb", shuffle=False)

Found 12709 images belonging to 4 classes.
Found 1413 images belonging to 4 classes.


In [5]:
# Neural network
from keras_applications.efficientnet import EfficientNetB3

base_model = EfficientNetB3(include_top=False,
                   weights='imagenet',
                   input_tensor=None,
                   input_shape=(img_size, img_size, 3),
                   pooling=None,
                   backend=keras.backend, layers=keras.layers, models=keras.models, utils=keras.utils)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(4, activation='softmax')(x)
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# x = Flatten()(base.output)
# x = Dropout(0.5)(x)
# x = Dense(4, activation='softmax')(x)
# model = Model(inputs=base.inputs, outputs=x)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [6]:
# Set optimizer and loss function
opt_adam = keras.optimizers.Adam(lr=0.0001, amsgrad=True)
model.compile(loss='categorical_crossentropy',
            optimizer=opt_adam,
            metrics=['categorical_accuracy'])

In [7]:
# callbacks
tensorboard = TensorBoard(log_dir="logs/Recyclable-{}-EfficientNetB3-bs32".format(datetime.now().isoformat()[:19]))
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.1, patience=5, min_lr=0.00000001, min_delta=1e-2)
checkpoint = keras.callbacks.ModelCheckpoint('logs/Recyclable-bs32-weights.{epoch:02d}-{val_categorical_accuracy:.3f}-DenseNet169.hdf5', monitor='val_categorical_accuracy', 
        verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=1)

In [ ]:
# Fit the model!
model.fit_generator(trainGenerator,
            steps_per_epoch = 400,  # trashnet 60
            epochs = 30,
            validation_data = validationGenerator,
            validation_steps = 40,  
            callbacks=[tensorboard, reduce_lr, checkpoint])




Epoch 1/30
400/400 [==============================] - 468s 1s/step - loss: 0.1571 - categorical_accuracy: 0.9494 - val_loss: 0.0180 - val_categorical_accuracy: 0.9961


Epoch 00001: val_categorical_accuracy improved from -inf to 0.99609, saving model to logs/Recyclable-bs32-weights.01-0.996-DenseNet169.hdf5
Epoch 2/30
400/400 [==============================] - 436s 1s/step - loss: 0.0221 - categorical_accuracy: 0.9930 - val_loss: 3.3672e-04 - val_categorical_accuracy: 0.9968

Epoch 00002: val_categorical_accuracy improved from 0.99609 to 0.99681, saving model to logs/Recyclable-bs32-weights.02-0.997-DenseNet169.hdf5
Epoch 3/30
400/400 [==============================] - 435s 1s/step - loss: 0.0160 - categorical_accuracy: 0.9956 - val_loss: 7.9943e-06 - val_categorical_accuracy: 0.9984

Epoch 00003: val_categorical_accuracy improved from 0.99681 to 0.99840, saving model to logs/Recyclable-bs32-weights.03-0.998-DenseNet169.hdf5
Epoch 4/30
400/400 [==============================] - 432s